In [1]:
import importlib
import pipeline
import feature_enginier
import datasets
importlib.reload(pipeline)
importlib.reload(datasets)
importlib.reload(feature_enginier)
print("import required things")
run_id="9c890f2cc0a843188a93defe1cdae78d"

import required things


In [2]:
import mlflow
from torch.utils.data import RandomSampler, DataLoader
import torch
import feature_enginier
import pandas as pd

importlib.reload(feature_enginier)


class ModelWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        config = {
            "epochs": 1,
            "lr": 0.001,
            "batch_size": 4000,
            "device": "cpu",
            "model_name": "res wide deep",
            "run id": run_id,
            'data address': "train_dataset.csv",
            'train pure address': "data/train_pure_dataset.pickle",
            'dev pure address': "data/dev_pure_dataset.pickle",
            'test pure address': "data/test_pure_dataset.pickle",
            'train clean address': "data/train_clean_dataset.pickle",
            'dev clean address': "data/dev_clean_dataset.pickle",
            'test clean address': "data/test_clean_dataset.pickle"
        }
        all_data = pd.read_csv(config['data address'])
        n = len(all_data)
        # set dataset sizes / train_set:80%, validation_set:10%, test_set:10%
        train_size = int(n * 0.80)
        # build datasets
        train_data = all_data.iloc[:train_size]
        self.feature_builder = feature_enginier.feature_engineering(data=train_data)
        self.feature_builder.clean_function() # type: feature_enginier.feature_engineering
        self.model = pipeline.Pipeline.load_model(config)
        self.model.eval()


    def predict(self, context, input_model):
        self.feature_builder.change_data(input_model)
        self.feature_builder.clean_function()
        dataset = datasets.RecomenderTestingDataset(input_model)
        dataloader = DataLoader(
            dataset,
            batch_size=1,
            num_workers=0,
            drop_last=False,
            pin_memory=True)
        outputs = []
        with torch.no_grad():
            self.model.eval()
            for step, x in enumerate(dataloader):
                # forward pass
                output = self.model(x)
                outputs.append(output)

        outputs = (torch.cat(outputs) > 0.5).squeeze().numpy()
        return pd.DataFrame.from_dict(outputs)


In [5]:
import numpy as np
import sklearn
import os
conda_env = {
    'channels': ['defaults'],
    'dependencies': [
            f'python=3.9.7',
      {
          'pip':[
            f'mlflow=={mlflow.__version__}',
            f'torch=={torch.__version__}',
            f'numpy=={np.__version__}',
            f'sklearn == {sklearn.__version__}'
    ]
      }
    ],
    'name': 'Recommendation'
}
with mlflow.start_run() as run:
    python_model=ModelWrapper()
    mlflow.log_param("try", 4)
    mlflow.pyfunc.log_model(artifact_path ="data/Recommendation",python_model=python_model, conda_env=conda_env)
    os.system(f'mlflow models serve -m "./mlruns/0/{run.info.run_id}/artifacts/data/Recommendation" --no-conda -h 127.0.0.1 -p 8080')

In [ ]:
import os
run_id = "9c890f2cc0a843188a93defe1cdae78d"
os.system(f'mlflow models serve -m "./mlruns/0/{run_id}/artifacts/model" --no-conda -h 127.0.0.1 -p 8080')

In [5]:
import mlflow

with mlflow.start_run() as run:
    config = {
        "epochs": 1,
        "lr": 0.001,
        "batch_size": 4000,
        "device": "cpu",
        "model_name": "res wide deep",
        "run id": run.info.run_id,

        'data address': "train_dataset.csv",
        'train pure address': "data/train_pure_dataset.pickle",
        'dev pure address': "data/dev_pure_dataset.pickle",
        'test pure address': "data/test_pure_dataset.pickle",
        'train clean address': "data/train_clean_dataset.pickle",
        'dev clean address': "data/dev_clean_dataset.pickle",
        'test clean address': "data/test_clean_dataset.pickle",
    }

In [ ]:
config = {
        "epochs": 1,
        "lr": 0.001,
        "batch_size": 4000,
        "device": "cpu",
        "model_name": "res wide deep",
        "run info": run.info.run_id,

        'data address': "train_dataset.csv",
        'train pure address': "data/train_pure_dataset.pickle",
        'dev pure address': "data/dev_pure_dataset.pickle",
        'test pure address': "data/test_pure_dataset.pickle",
        'train clean address': "data/train_clean_dataset.pickle",
        'dev clean address': "data/dev_clean_dataset.pickle",
        'test clean address': "data/test_clean_dataset.pickle",
    }


In [ ]:
print("start pipe line")
pipeline.Pipeline.get_data(config)
print("finish getting data/ start preprocessing data")
pipeline.Pipeline.preprocess_data(config)
print("finish preprocessing data/ start training")
pipeline.Pipeline.train(config)
print("finish training/ start evaluation")
pipeline.Pipeline.evaluation(config)
print("finish evaluation/ start deploy")
pipeline.Pipeline.deploy(config)